In [140]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)
session.headers['User-agent'] = 'n.finance/1.0'

In [141]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

stocks = ['2330.TW', '2882.TW']
tickers = yf.Tickers(stocks, session=session)

In [148]:
tickers.history(start='2015-01-01', end='2016-12-31', auto_adjust=False)

[*********************100%%**********************]  2 of 2 completed


Adj Close              Close            Dividends          \
               2330.TW    2882.TW 2330.TW    2882.TW   2330.TW 2882.TW   
Date                                                                     
2015-01-05  106.628204  31.101538   139.5  46.188869       0.0     0.0   
2015-01-06  102.042023  30.129614   133.5  44.745468       0.0     0.0   
2015-01-07  102.424202  30.163128   134.0  44.795238       0.0     0.0   
2015-01-08  105.481659  30.531790   138.0  45.342739       0.0     0.0   
2015-01-09  102.424202  30.665848   134.0  45.541828       0.0     0.0   
...                ...        ...     ...        ...       ...     ...   
2016-12-26  147.117401  35.340591   179.5  47.831364       0.0     0.0   
2016-12-27  147.527191  35.009621   180.0  47.383411       0.0     0.0   
2016-12-28  149.166382  35.781891   182.0  48.428631       0.0     0.0   
2016-12-29  147.527191  35.230270   180.0  47.682045       0.0     0.0   
2016-12-30  148.756577  35.450916   181.5  47.980679       0.0     0.0   

              High                Low               Open             \
           2330.TW    2882.TW 2330.TW    2882.TW 2330.TW    2882.TW   
Date                                                                  
2015-01-05   140.5  46.537277   137.5  45.989780   140.5  46.537277   
2015-01-06   137.5  45.740917   133.0  44.596149   137.5  45.740917   
2015-01-07   135.0  45.093876   133.5  44.645924   133.5  44.745468   
2015-01-08   138.0  45.691147   136.0  45.143646   136.5  45.492054   
2015-01-09   135.5  45.890236   133.0  45.492054   135.0  45.890236   
...            ...        ...     ...        ...     ...        ...   
2016-12-26   180.0  47.831364   179.0  46.885685   179.0  46.985229   
2016-12-27   180.5  47.831364   179.5  47.283867   180.0  47.831364   
2016-12-28   182.0  48.577950   180.5  47.433182   181.0  47.682045   
2016-12-29   181.5  48.577950   180.0  47.433182   180.0  48.229542   
2016-12-30   182.5  48.329086   180.0  47.831364   181.0  47.930908   

           Stock Splits            Volume            
                2330.TW 2882.TW   2330.TW   2882.TW  
Date                                                 
2015-01-05          0.0     0.0  32046000  15562813  
2015-01-06          0.0     0.0  66778000  39533887  
2015-01-07          0.0     0.0  43703000  16251949  
2015-01-08          0.0     0.0  42491000  15956605  
2015-01-09          0.0     0.0  61558000  10060778  
...                 ...     ...       ...       ...  
2016-12-26          0.0     0.0   5675000   7701041  
2016-12-27          0.0     0.0   8252000   7592547  
2016-12-28          0.0     0.0  21226000  13916322  
2016-12-29          0.0     0.0  30200000  18946209  
2016-12-30          0.0     0.0  36610000   9481141  

[490 rows x 16 columns]

In [142]:
df = pd.DataFrame()

for stock_code in stocks:
    price = tickers.tickers[stock_code].history(start='2015-01-01', end='2016-12-31', auto_adjust=False)
    simpleret = price['Close'].pct_change().dropna()
    simpleret.index = simpleret.index.date  # type: ignore
    simpleret.name = stock_code

    df = pd.concat([df, simpleret], axis=1)

In [143]:
df

,2330.TW,2882.TW
2015-01-06,-0.043011,-0.031250
2015-01-07,0.003745,0.001112
2015-01-08,0.029851,0.012222
2015-01-09,-0.028986,0.004391
2015-01-12,-0.014925,-0.010929
...,...,...
2016-12-26,0.005602,0.022340
2016-12-27,0.002786,-0.009365
2016-12-28,0.011111,0.022059
2016-12-29,-0.010989,-0.015416


In [144]:
df.describe()

,2330.TW,2882.TW
count,489.000000,489.000000
mean,0.000657,0.000220
std,0.015417,0.016884
min,-0.051370,-0.092873
25%,-0.008571,-0.008174
50%,0.000000,0.000000
75%,0.009434,0.008264
max,0.073913,0.069284


## 投資組合的報酬

- 由一種以上的證券或資產所構成的集合，稱為投資組合。
- 投資組合的預期報酬率
    - 為所有個別資產預期報酬率的加權平均數。

$
\begin{aligned}
R_P &= 總收益/初始投入資本 \\
    &= \frac{(W_0 \omega_A R_A)+(W_0 \omega_B R_B)}{W_0} \\
    &= \omega_A R_A + \omega_B R_B \\
    &= \sum{\omega R}
\end{aligned}
$

$\omega_i$ 表示各資產的權重，$R_i$ 表示投資各資產所能獲得的預期報酬率。

In [145]:
df["2330.TW"]

2015-01-06   -0.043011
2015-01-07    0.003745
2015-01-08    0.029851
2015-01-09   -0.028986
2015-01-12   -0.014925
                ...   
2016-12-26    0.005602
2016-12-27    0.002786
2016-12-28    0.011111
2016-12-29   -0.010989
2016-12-30    0.008333
Name: 2330.TW, Length: 489, dtype: float64

In [146]:
# 以 50% 資金投資台積電，以 50% 資金投資
# 國泰金，試著算出投資組合的報酬。

weight = 0.5
portfolio_ret = weight * df["2330.TW"].mean() + (1 - weight) * df["2882.TW"].mean()
portfolio_ret

0.00043851117709123224

## 投資組合的風險

- 以標準差或變異數衡量投資組合的風險。

$
\begin{aligned}
&Var(W_1 R_1 + W_2 R_2) \\
= &W_1^2 Var(R_1) + W_2^2 Var(R_2) + 2 W_1 W_2 Cov(R_1, R_2) \\
= &W_1^2 \sigma_1^2 + W_2^2 \sigma_2^2 + 2 W_1 W_2 \sigma_1 \sigma_2 \rho_{1,2}
\end{aligned}
$

- 由多種資產構成的投資組合不但包含原先個別資產的風險（有權數的調整），尚隱 含個別資產間相互影響所帶來的風險。

In [147]:
# 以 50% 資金投資台積電，以 50% 資金投資
# 國泰金，試著算出投資組合的風險。

import math


weight = 0.5
twse, cfl = df["2330.TW"], df["2882.TW"]

portfolio_var = (
    (weight**2) * (twse.std()**2) + ((1-weight)**2) *
                  (cfl.std()**2) + 2 * weight * (1-weight) *
                  twse.corr(cfl, method="pearson") * twse.std() * cfl.std()
)
math.sqrt(portfolio_var)

0.01374322326267572